In [1]:
import os
import json

from pymongo import MongoClient

# Prep

Set environment variables and setup simulated vasp run

In [2]:
os.environ["VASP_PSP_DIR"] = os.path.join(os.getcwd(), "reference_files")

# Structure and input set

In [3]:
from pymatgen.util.testing import PymatgenTest
from pymatgen.io.vasp.sets import MPRelaxSet

struct_si = PymatgenTest.get_structure("Si")
vis = MPRelaxSet(struct_si, force_gamma=True)

# Define Fireworks and Workflow

In [4]:
from fireworks import Workflow
from matmethods.vasp.fireworks.core import OptimizeFW, StaticFW, NonSCFFW

fw1 = OptimizeFW(struct_si, vasp_input_set=vis, vasp_cmd=None, db_file=">>db_file<<")
fw2 = StaticFW(struct_si, vasp_input_set=vis, vasp_cmd=None, db_file=">>db_file<<", parents=[fw1])
fw3 = NonSCFFW(struct_si, vasp_cmd=None, db_file=">>db_file<<", parents=[fw2], mode="uniform")
fw4 = NonSCFFW(struct_si, vasp_cmd=None, db_file=">>db_file<<", parents=[fw2], mode="line")

fws = [fw1, fw2, fw3, fw4]

wf = Workflow(fws)

In [5]:
from matmethods.vasp.vasp_powerups import use_fake_vasp

reference_dir = os.path.join(os.getcwd(), "reference_files")
si_ref_dirs = {"structure optimization": os.path.join(reference_dir, "Si_structure_optimization"),
             "static": os.path.join(reference_dir, "Si_static"),
             "nscf uniform": os.path.join(reference_dir, "Si_nscf_uniform"),
             "nscf line": os.path.join(reference_dir, "Si_nscf_line")}

wf_fake = use_fake_vasp(wf, si_ref_dirs)

# Connect to Launchpad and reset

In [6]:
import datetime
from fireworks import LaunchPad

lp = LaunchPad.from_file("my_launchpad.yaml")
lp.reset(datetime.datetime.now().strftime('%Y-%m-%d'))

2016-08-02 15:08:09,355 INFO Performing db tune-up
2016-08-02 15:08:09,452 INFO LaunchPad was RESET.


# Submit Workflow

In [7]:
lp.add_wf(wf_fake)

2016-08-02 15:08:10,916 INFO Added a workflow. id_map: {-1: 4, -4: 1, -3: 2, -2: 3}


{-4: 1, -3: 2, -2: 3, -1: 4}

# Run

In [8]:
from fireworks import FWorker
from fireworks.core.rocket_launcher import rapidfire

db_file = os.path.join(os.getcwd(), "db.json")
rapidfire(lp, fworker=FWorker(env={"db_file": db_file}))

2016-08-02 15:08:13,286 INFO Created new dir /home/jovyan/work/launcher_2016-08-02-15-08-13-286465
2016-08-02 15:08:13,288 INFO Launching Rocket
2016-08-02 15:08:13,376 INFO RUNNING fw_id: 4 in directory: /home/jovyan/work/launcher_2016-08-02-15-08-13-286465
2016-08-02 15:08:13,382 INFO Task started: {{matmethods.vasp.firetasks.write_inputs.WriteVaspFromIOSet}}.
2016-08-02 15:08:13,400 INFO Task completed: {{matmethods.vasp.firetasks.write_inputs.WriteVaspFromIOSet}} 
2016-08-02 15:08:13,401 INFO Task started: {{matmethods.vasp.firetasks.run_calc.RunVaspFake}}.
2016-08-02 15:08:13,430 INFO matmethods.vasp.firetasks.run_calc RunVaspFake: verified inputs successfully
2016-08-02 15:08:13,443 INFO matmethods.vasp.firetasks.run_calc RunVaspFake: ran fake VASP, generated outputs
2016-08-02 15:08:13,445 INFO Task completed: {{matmethods.vasp.firetasks.run_calc.RunVaspFake}} 
2016-08-02 15:08:13,446 INFO Task started: {{matmethods.common.firetasks.glue_tasks.PassCalcLocs}}.
2016-08-02 15:08:13

# Retrieve and Verify

In [9]:
def get_task_database(db_file):
    """
    connect to the database
    """
    with open(db_file) as f:
        creds = json.loads(f.read())
        conn = MongoClient(creds["host"], creds["port"])
        db = conn[creds["database"]]
        if "admin_user" in creds:
            db.authenticate(creds["admin_user"], creds["admin_password"])
        return db
        
def get_task_collection(db_file):
    """
    connect to the database and return task collection
    """
    with open(db_file) as f:
        creds = json.loads(f.read())
        db = get_task_database(db_file)
        return db[creds["collection"]]

In [10]:
from pymongo import DESCENDING

d1 = get_task_collection(db_file).find_one({"task_label": "structure optimization"}, sort=[("_id", DESCENDING)])

d2 = get_task_collection(db_file).find_one({"task_label": "static"}, sort=[("_id", DESCENDING)])

d3 = get_task_collection(db_file).find_one({"task_label": "nscf uniform"}, sort=[("_id", DESCENDING)])

d4 = get_task_collection(db_file).find_one({"task_label": "nscf line"}, sort=[("_id", DESCENDING)])

In [11]:
d1["output"]["bandgap"] # 0.65

0.6500000000000004

# Debug

In [12]:
!rm -rf launcher

In [ ]:
from monty.os.path import zpath
import os
from pymatgen.io.vasp.inputs import get_potcar_dir, PotcarSingle

symbol="Si"
funcdir = PotcarSingle.functional_dir["PBE"]
d = get_potcar_dir()
print(d)
paths_to_try = [os.path.join(d, funcdir, "POTCAR.{}".format(symbol)),
                        os.path.join(d, funcdir, symbol, "POTCAR")]
for p in paths_to_try:
            p = os.path.expanduser(p)
            print("0",p)
            p = zpath(p)
            print("1",p)
            if os.path.exists(p):
                print("2",p)

In [ ]:
os.path.exists('/project/projectdirs/matgen/POTCARs/POT_GGA_PAW_PBE/POTCAR.Si.gz')

In [ ]:
os.listdir("/home/jovyan/")